In [1]:
import pickle
import numpy as np
import pandas as pd

In [3]:
# with open("data_storage/small_dataset.obj", "rb") as file:
#     data = pickle.load(file)

# If you want to use main data download it from here: https://zenodo.org/record/7437768#.Y6W6YnaZO5d
# Then move the downloaded file to the data_storage folder
with open("data_storage/Extended_manhattan_pooling.obj", "rb") as file:
    data = pickle.load(file)

In [6]:
data[0]["requests"][:3]

,id,pickup_datetime,dropoff_datetime,trip_duration,origin,destination,status,dist,treq,ttrav,...,u_PT,ride_id,ttrav_sh,u_sh,kind,position,x_org,y_org,x_dest,y_dest
0,id0190469,2016-01-01 00:00:17,2016-01-01 00:14:26,849,42427352,42438018,0,14340,0,2390,...,999999,306,2591,29.772825,21,0,-73.981942,40.719462,-73.938959,40.828984
1,id1210365,2016-01-01 00:01:01,2016-01-01 00:07:49,408,42432756,42434113,0,2640,44,440,...,999999,1,440,5.500000,1,0,-73.965196,40.801107,-73.947489,40.815207
2,id3991965,2016-01-01 00:20:08,2016-01-01 00:29:16,548,42437917,42434113,0,2705,1191,450,...,999999,118,450,5.632500,1,0,-73.963370,40.798568,-73.947489,40.815207


In [7]:
concatenated_data = pd.concat([d["requests"] for d in data])

In [8]:
concatenated_data[:3]

,id,pickup_datetime,dropoff_datetime,trip_duration,origin,destination,status,dist,treq,ttrav,...,u_PT,ride_id,ttrav_sh,u_sh,kind,position,x_org,y_org,x_dest,y_dest
0,id0190469,2016-01-01 00:00:17,2016-01-01 00:14:26,849,42427352,42438018,0,14340,0,2390,...,999999,306,2591,29.772825,21,0,-73.981942,40.719462,-73.938959,40.828984
1,id1210365,2016-01-01 00:01:01,2016-01-01 00:07:49,408,42432756,42434113,0,2640,44,440,...,999999,1,440,5.500000,1,0,-73.965196,40.801107,-73.947489,40.815207
2,id3991965,2016-01-01 00:20:08,2016-01-01 00:29:16,548,42437917,42434113,0,2705,1191,450,...,999999,118,450,5.632500,1,0,-73.963370,40.798568,-73.947489,40.815207


In [10]:
len(concatenated_data)

4908349

In [11]:
import h3
import folium
from geojson.feature import Feature, FeatureCollection
import json
import osmnx as ox

In [12]:
# add hexagon column to the DataFrame
df = concatenated_data
zoning_level = 9
df['h3_address'] = df.apply(lambda row: h3.geo_to_h3(row.y_org, row.x_org, zoning_level), axis=1)
# df

In [13]:
# Here are some visualization functions to check the hexagons without colorbar.

def visualize_hexagons(hexagons, color="red", w=1, folium_map=None):
    """
    hexagons is a list of hexcluster. Each hexcluster is a list of hexagons. 
    eg. [[hex1, hex2], [hex3, hex4]]
    """
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=w,color=color,fill= True)
        m.add_child(my_PolyLine)
    return m
    

def visualize_polygon(polyline, color):
    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
    m.add_child(my_PolyLine)
    return m

In [14]:
# Using the above function you can visualize the hexagons.

hexagons = list(df.h3_address.unique())
visualize_hexagons(hexagons, color="red", w=1, folium_map=None)

In [15]:
# to plot the KPIs you need to groupby the requests by their hexagon address
# according to you purpose you may use groupby().mean() or groupby().sum()

df_new = df.groupby(['h3_address']).mean()
df_new.reset_index(inplace=True)
df_new

,h3_address,trip_duration,origin,destination,status,dist,treq,ttrav,VoT,delta,...,u_PT,ride_id,ttrav_sh,u_sh,kind,position,x_org,y_org,x_dest,y_dest
0,892a1008803ffff,847.052632,4.193894e+08,4.488115e+08,0.0,3802.157895,774.105263,633.263158,0.0035,179.761596,...,999999.0,153.947368,665.592105,7.762174,6.868421,0.039474,-73.974041,40.797709,-73.978387,40.775798
1,892a1008807ffff,725.584052,4.215707e+08,5.837664e+08,0.0,3093.157321,885.581536,515.114606,0.0035,146.268554,...,999999.0,162.166966,532.420681,6.298150,7.477774,0.168943,-73.971965,40.794979,-73.967005,40.788113
2,892a100880fffff,824.166667,1.013633e+09,1.375909e+08,0.0,3598.733333,864.650000,599.350000,0.0035,170.151319,...,999999.0,198.950000,625.045833,7.324879,8.458333,0.150000,-73.975798,40.795267,-73.972541,40.780451
3,892a1008813ffff,853.807692,2.332930e+08,2.673655e+08,0.0,3941.576923,804.615385,656.500000,0.0035,186.349451,...,999999.0,181.134615,672.375000,8.020398,9.230769,0.153846,-73.971639,40.800187,-73.972202,40.782748
4,892a1008817ffff,760.195462,2.339739e+08,5.675582e+08,0.0,3205.039376,881.763244,533.754401,0.0035,151.585294,...,999999.0,168.267540,553.513306,6.528457,7.802453,0.134479,-73.970362,40.797762,-73.971382,40.782189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,892a1072dcbffff,764.835239,1.377229e+09,4.806957e+08,0.0,3370.110289,930.325488,561.302623,0.0035,159.410166,...,999999.0,171.439812,586.110289,6.845400,7.388702,0.063215,-73.992977,40.713153,-73.988825,40.734628
528,892a1072dcfffff,825.750000,1.581471e+08,5.065152e+08,0.0,3639.555556,855.888889,606.250000,0.0035,172.058059,...,999999.0,174.958333,629.784722,7.372075,7.548611,0.076389,-73.991116,40.710217,-73.987408,40.732481
529,892a1072dd3ffff,850.778571,4.244319e+07,4.602543e+08,0.0,3265.300000,938.078571,543.821429,0.0035,154.385604,...,999999.0,190.887500,567.308929,6.641501,7.419643,0.066071,-73.987370,40.715774,-73.988686,40.734778
530,892a1072dd7ffff,742.320000,1.297974e+08,5.557701e+07,0.0,3240.560000,1020.600000,539.760000,0.0035,153.202286,...,999999.0,170.160000,553.370000,6.613922,5.310000,0.050000,-73.984647,40.713785,-73.989283,40.730289


In [16]:
# use this function to get the plots with colorbar.
# adjust the bins before plotting. Otherwise you will see an error.
# you may also modify the other features like colors, opacity and line_weight. 

def h3_visualization(df, hex_col, value_col):
    df['lat'] = df[hex_col].apply(lambda x: h3.h3_to_geo(x)[0])
    df['lng'] = df[hex_col].apply(lambda x: h3.h3_to_geo(x)[1])
    # df.set_index('hex_address', inplace=True)
    df['geom'] = df.apply(lambda x: {"type": "Polygon","coordinates": [h3.h3_to_geo_boundary(h = x[hex_col], geo_json = True)]}, axis = 1)
    list_features = []
    for i, row in df.iterrows():
            feature = Feature(geometry = row["geom"],
                              id = row[hex_col],
                              properties = {"resolution": 9})
            list_features.append(feature)
    list_features 

    feat_collection = FeatureCollection(list_features)
    geojson_hexes = json.dumps(feat_collection)
    geojson_hexes

    # cx, cy = G.nodes[centeral_node]['x'], G.nodes[centeral_node]['y']
    # CENTER = [cy, cx]
    CENTER = [h3.h3_to_geo(df.loc[0][hex_col])[0], h3.h3_to_geo(df.loc[0][hex_col])[1]]

    tile = 'cartodbpositron'
    base_map = folium.Map(zoom_start=11,tiles=tile, zoomControl =  False, location=CENTER)
    
    step = (df[value_col].max() - df[value_col].min())/5
    print(df[value_col].min(),'  ', df[value_col].max()+step, '  ', step)
    bins = list(np.arange(df[value_col].min(),df[value_col].max()+step,step))
    # bins = list(np.arange(0, 100000, 20000))
    # bins = [0,20000,40000,60000,80000,1000000]

    # for colors see https://user-images.githubusercontent.com/17128994/115975254-c2031e00-a56b-11eb-8025-d82d36bfda1d.png
    color_set = ["Spectral", "RdYlGn", "PuBu", "Accent", "OrRd", "Set1", "Set2", "Set3", "BuPu", "Dark2", "RdBu", "Oranges", "BuGn", "PiYG", "YlOrBr", "YlGn", "Pastel2", "RdPu", "Greens", "PRGn", "YlGnBu", "RdYlBu", "Paired", "BrBG", "Purples", "Reds", "Pastel1", "GnBu", "Greys", "RdGy", "YlOrRd", "PuOr", "PuRd", "Blues", "PuBuGn"]
    m = folium.Choropleth(geo_data = geojson_hexes,data = df, columns = [hex_col,value_col],key_on ="feature.id",
            fill_color='Reds', control = False, bins = bins, line_weight=1, legend_name='',
            fill_opacity=0.5, line_opacity=0.2).add_to(base_map)

    return base_map

In [17]:
h3_visualization(df_new, 'h3_address', 'dist')

0.0    120000.0    20000.0


In [18]:
# to get a plot for number of request for each hexagon I add a column of ones and 
# then use groupby().sum() But you may use another method

df['number_of_req'] = 1
df_new = df.groupby(['h3_address']).sum()
df_new.reset_index(inplace=True)
df_new

,h3_address,trip_duration,origin,destination,status,dist,treq,ttrav,VoT,delta,...,ride_id,ttrav_sh,u_sh,kind,position,x_org,y_org,x_dest,y_dest,number_of_req
0,892a1008803ffff,64376,31873590620,34109677692,0,288964,58832,48128,0.2660,1.366188e+04,...,11700,50585,589.925225,522,3,-5.622027e+03,3100.625858,-5.622357e+03,3098.960656,76
1,892a1008807ffff,12111449,7036858381327,9744227965103,0,51630982,14782127,8598293,58.4220,2.441515e+06,...,2706891,8887166,105128.720380,124819,2820,-1.234740e+06,680949.793197,-1.234657e+06,680835.188367,16692
2,892a100880fffff,197800,243271982064,33021815508,0,863696,207516,143844,0.8400,4.083632e+04,...,47748,150011,1757.970855,2030,36,-1.775419e+04,9790.864017,-1.775341e+04,9787.308266,240
3,892a1008813ffff,88796,24262470248,27806014512,0,409924,83680,68276,0.3640,1.938034e+04,...,18838,69927,834.121400,960,16,-7.693050e+03,4243.219463,-7.693109e+03,4241.405843,104
4,892a1008817ffff,9112463,2804644651085,6803320611803,0,38418807,10569696,6398114,41.9545,1.817053e+06,...,2017023,6634964,78256.613935,93528,1612,-8.866827e+05,489042.771401,-8.866950e+05,488856.103039,11987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,892a1072dcbffff,1137310,2047939623686,714794487118,0,5011354,1383394,834657,5.2045,2.370429e+05,...,254931,871546,10179.110110,10987,94,-1.100276e+05,60540.458633,-1.100214e+05,60572.392414,1487
528,892a1072dcfffff,118908,22773178652,72938193628,0,524096,123248,87300,0.5040,2.477636e+04,...,25194,90689,1061.578810,1087,11,-1.065472e+04,5862.271184,-1.065419e+04,5865.477210,144
529,892a1072dd3ffff,476436,23768187260,257742388540,0,1828568,525324,304540,1.9600,8.645594e+04,...,106897,317693,3719.240385,4155,37,-4.143293e+04,22800.833300,-4.143366e+04,22811.475904,560
530,892a1072dd7ffff,74232,12979741016,5557700756,0,324056,102060,53976,0.3500,1.532023e+04,...,17016,55337,661.392150,531,5,-7.398465e+03,4071.378482,-7.398928e+03,4073.028950,100


In [19]:
h3_visualization(df_new, 'h3_address', 'number_of_req')

4    111240.4    18539.4
